# Lyrics detector Challenge

The goal for this challenge is to leverage your knowledge of Deep Learning to design and train a lyrics classifier. For a given verse $X$, our model should learn to predict the artist $y$. The dataset consists of lyrics scrapped from the Genius website.

### Objectives:
- Text preprocessing
- Text embedding
- Train a RNN to detect the artist behind a set of lyrics

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf

## 1. Data Cleaning

Our dataset contains around 4,000 verses of lyrics from different artists: Drake, Ed Sheeran and Kanye West (the verses are given in this order).

In [2]:
raw_data = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/verses.csv")
data = raw_data.copy() # From now on, update `data` as you see fit and don't touch raw_data
data

artist                                              verse
0          Drake                            Ayy, woah Ayy, ayy Yeah
1          Drake  I'm makin' a change today The liquor been taki...
2          Drake  I can't just be with you and only you Yeah, I ...
3          Drake  Well, summer, all I did was rest, okay? And Ne...
4          Drake  I'm makin' a change today The liquor been taki...
...          ...                                                ...
3970  Kanye West  Dame was like, "Yo you got a deal with Capitol...
3971  Kanye West  Then one day I just went ahead and played it, ...
3972  Kanye West             "I ain't gonna front, it's kinda hot."
3973  Kanye West  Like they still weren't looking at me like a r...
3974  Kanye West  "You gotta be under an umbrella, you'll get ra...

[3975 rows x 2 columns]

❓ **Have a look at the verse index 18th**. 
- What do you observe?
- Clean verses from non standard characters using [`unidecode.unidecode()`](https://pypi.org/project/Unidecode/)

In [3]:
data.iloc[18,1]

'Ayy, yeah Pipe this shit up and I turn this shit up When I was down, I\u2005was\u2005down Now them diamonds\u2005for fun like damn Ayy, yeah'

In [4]:
from unidecode import unidecode

In [5]:
data['verse'] = data['verse'].apply(unidecode)

In [6]:
data.iloc[18,1]

'Ayy, yeah Pipe this shit up and I turn this shit up When I was down, I was down Now them diamonds for fun like damn Ayy, yeah'

❓ **Check if some verses are duplicated.** 
- It can be frequent in music lyrics.
- If so, remove them to avoid data leaks between train and test sets

In [7]:
data = data.drop_duplicates()

In [8]:
data.shape

(3031, 2)

In [9]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'data_loading',
    shape=data.shape,
    verses=data.verse[:50]
)

result.write()

## 2. Data Analysis (given to you)

👉 **We check the number of unique artist and the number of verses per artist**

In [10]:
data.artist.value_counts()

Drake         1391
Ed Sheeran     861
Kanye West     779
Name: artist, dtype: int64

👉 **For each artist, let's have a look at the top-10 most used words to see if they look similar?**

We'll use Tensorflow's [`Tokenizer`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer)'s index_word

In [11]:
drake = data[data.artist =='Drake'].verse
ed = data[data.artist =='Ed Sheeran'].verse
kanye = data[data.artist =='Kanye West'].verse

In [12]:
tokenizer_drake = tf.keras.preprocessing.text.Tokenizer()
tokenizer_ed = tf.keras.preprocessing.text.Tokenizer()
tokenizer_kanye = tf.keras.preprocessing.text.Tokenizer()

tokenizer_drake.fit_on_texts(drake)
tokenizer_ed.fit_on_texts(ed)
tokenizer_kanye.fit_on_texts(kanye)

In [13]:
pd.DataFrame(data={
    "Drake": pd.Series(tokenizer_drake.index_word)[:10],
    "Ed Sheeran": pd.Series(tokenizer_ed.index_word)[:10],
    "Kanye West": pd.Series(tokenizer_kanye.index_word)[:10],
})

Drake Ed Sheeran Kanye West
1      i          i          i
2    you        you        the
3    the        the        you
4    and        and        and
5     to         me         to
6      a         my          a
7     it         to         my
8     me          a         it
9    i'm         in         me
10    my        i'm         in

👉 **Let's quantify how much vocabulary do they have in common**

- An artist **vocabulary** is the **set** of all unique used words
- We compute the `ratio` of (i) the length of vocabulary they **share**, over (ii) the length of the **total** vocabulary of the dataset

<details>
    <summary>Hints</summary>

We'll use Python [`set.intersection()`](https://www.programiz.com/python-programming/methods/set/intersection) and [`set.union()`](https://www.programiz.com/python-programming/methods/set/union)
</details>

In [14]:
drake_vocabulary = set(tokenizer_drake.index_word.values())
ed_vocabulary = set(tokenizer_ed.index_word.values())
kanye_vocabulary = set(tokenizer_kanye.index_word.values())

In [15]:
common_vocabulary = drake_vocabulary.intersection(ed_vocabulary).intersection(kanye_vocabulary)
global_vocabulary = drake_vocabulary.union(ed_vocabulary).union(kanye_vocabulary)

ratio = len(common_vocabulary)/len(global_vocabulary)
print(f"{ratio*100:.2f}% of the artists' vocabulary is common")

15.28% of the artists' vocabulary is common


## 3. Data Preprocessing

### 3.1 Word Embedding
We now need to think about embedding our sentences into numbers. We will be using [`gensim.models.Word2Vec`](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec) to embed each word of the sentence and concatenate the embeddings of the words forming the sentence.

❓ **Transform the list of strings (verses) into a list of word sequences (a word sequence is a list of words contained in a string)**
- Store these sequences of words in a new column `data["seq"]` in your dataframe
- You can use `tensorflow.keras.preprocessing.text.text_to_word_sequence` 

In [16]:
data["seq"] = data['verse'].apply(tf.keras.preprocessing.text.text_to_word_sequence)

<ipython-input-16-fbe2dfa34882>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["seq"] = data['verse'].apply(tf.keras.preprocessing.text.text_to_word_sequence)


In [17]:
data

artist                                              verse  \
0          Drake                            Ayy, woah Ayy, ayy Yeah   
1          Drake  I'm makin' a change today The liquor been taki...   
2          Drake  I can't just be with you and only you Yeah, I ...   
3          Drake  Well, summer, all I did was rest, okay? And Ne...   
4          Drake  I'm makin' a change today The liquor been taki...   
...          ...                                                ...   
3970  Kanye West  Dame was like, "Yo you got a deal with Capitol...   
3971  Kanye West  Then one day I just went ahead and played it, ...   
3972  Kanye West             "I ain't gonna front, it's kinda hot."   
3973  Kanye West  Like they still weren't looking at me like a r...   
3974  Kanye West  "You gotta be under an umbrella, you'll get ra...   

                                                    seq  
0                           [ayy, woah, ayy, ayy, yeah]  
1     [i'm, makin', a, change, today, the, liquor, b...  
2     [i, can't, just, be, with, you, and, only, you...  
3     [well, summer, all, i, did, was, rest, okay, a...  
4     [i'm, makin', a, change, today, the, liquor, b...  
...                                                 ...  
3970  [dame, was, like, yo, you, got, a, deal, with,...  
3971  [then, one, day, i, just, went, ahead, and, pl...  
3972         [i, ain't, gonna, front, it's, kinda, hot]  
3973  [like, they, still, weren't, looking, at, me, ...  
3974  [you, gotta, be, under, an, umbrella, you'll, ...  

[3031 rows x 3 columns]

❓ **Let's check if we can cap the length of each sequences without losing too much information**
- Plot the distribution of sequences lengths using the [`seaborn.kdeplot`](https://seaborn.pydata.org/generated/seaborn.displot.html#seaborn-displot) function
- Does it seem reasonable to limit ourself to 300 words per verse later on? 

In [18]:
data['len_seq'] = data['seq'].apply(lambda x : len(x))

<ipython-input-18-56bd666644a7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['len_seq'] = data['seq'].apply(lambda x : len(x))


In [19]:
sns.displot(data=data, x='len_seq')

❓ **Keep only the first `300` words of each sequences to reduce the useless long tail of long verses**

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [21]:
data['seq'] = data['seq'].apply(lambda x : x[:300])

<ipython-input-21-311382ed50bb>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['seq'] = data['seq'].apply(lambda x : x[:300])


❓ **Train a `gensim.models.Word2Vec` model on your dataset** 
- You want to embed each word into vectors of dimension `100`
- No words should be excluded
- Give Word2Vec at least 50 epochs to be sure it converges
- Store these lists of vectors in a new column `data["embed"]`

In [23]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=data['seq'], vector_size=100, epochs=50)

/home/arthur/.pyenv/versions/3.8.6/envs/lewagon/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [24]:
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

In [25]:
data["embed"] = data['seq'].apply(lambda x : embed_sentence(word2vec, x))

<ipython-input-25-7608788501c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["embed"] = data['seq'].apply(lambda x : embed_sentence(word2vec, x))


In [26]:
data.head()

artist                                              verse  \
0  Drake                            Ayy, woah Ayy, ayy Yeah   
1  Drake  I'm makin' a change today The liquor been taki...   
2  Drake  I can't just be with you and only you Yeah, I ...   
3  Drake  Well, summer, all I did was rest, okay? And Ne...   
4  Drake  I'm makin' a change today The liquor been taki...   

                                                 seq  len_seq  \
0                        [ayy, woah, ayy, ayy, yeah]        5   
1  [i'm, makin', a, change, today, the, liquor, b...       60   
2  [i, can't, just, be, with, you, and, only, you...       96   
3  [well, summer, all, i, did, was, rest, okay, a...       65   
4  [i'm, makin', a, change, today, the, liquor, b...       61   

                                               embed  
0  [[-1.2272295, 0.47958648, 0.14716442, -1.26685...  
1  [[-0.046280082, -2.5213299, -1.5439209, 0.9487...  
2  [[1.0644122, 0.59299755, -0.09814267, 0.387671...  
3  [[0.52389437, -2.3491035, -1.2277962, 0.528269...  
4  [[-0.046280082, -2.5213299, -1.5439209, 0.9487...

In [27]:
# Check 
assert len(data['embed']) == len(data)

In [29]:
data.iloc[0,4].shape

(5, 100)

### 3.2 Create (X,y)

❓ **Create your numpy array `X` of shape (number_of_verses, 300, 100)**

- 300 words per verse (pad verses shorter than 300 with zeros at the end) 
- each words being a vector of size 100

<img src="https://raw.githubusercontent.com/lewagon/data-images/master/DL/padding.png" width=400>

In [35]:
X = pad_sequences(data['embed'], dtype='float32', padding='post', maxlen=300)

In [36]:
X.shape

(3031, 300, 100)

❓ **Create the numpy array `y` of shape `(n_verses, 3)` that contains the one-hot-encoded list of labels, for the RNN**

In [38]:
from sklearn.preprocessing import OneHotEncoder

In [39]:
ohe = OneHotEncoder(sparse = False)
ohe.fit(data[['artist']])
y = ohe.transform(data[['artist']])

In [40]:
y.shape

(3031, 3)

👉 We train/test split the dataset below for you

In [41]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [42]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'data_preprocessing',
    n_zeros = np.sum(X == 0),
    X_shape = X.shape,
    y_shape = y.shape,
)

result.write()

## 4. Recurrent Neural Network

👉 Run this code below if you haven't managed to build your own (X,Y) training sets. This will load them as solution

```python
! wget \
'https://wagon-public-datasets.s3.amazonaws.com/certification_france_2021_q2/data_lyrics_solution.pickle'

import pickle
with open("data_lyrics_solution.pickle", "rb") as file:
    (X_train, y_train, X_test, y_test) = pickle.load(file)
    
! rm data_lyrics_solution.pickle
```

❓ **First, store your baseline accuracy to beat as `score_baseline`**
- Consider predicting always the most frequent artist

In [52]:
data['artist'].value_counts()
score_baseline = (1391 / len(data))*100
print(f"score_baseline is {score_baseline} %")

score_baseline is 45.89244473771033 %


❓ **Create a RNN architecture to predict the artists `y`  given verses `X`** :

- Keep it simple: use only one LSTM layer and one *hidden* dense layer between the input and output layers
- Don't forget to take care of fake "zeros" added during preprocessing
- Store it into the `model` variable.

In [43]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

In [54]:
def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    return model

❓ **Train your `model` on the `(X_train, y_train)` training set**
- Use an appropriate loss
- Adapt the learning rate of your optimizer if convergence is too slow/fast
- Make sure your model does not overfit with appropriate control techniques

💡 You will not be judged by the computing power of your computer, you can reach decent performance in less than 3 minutes of training without GPUs.

In [55]:
model = init_model()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train, 
          batch_size = 32,
          epochs=100,
          validation_split=0.3,
          callbacks=[es],
          verbose=1
         )

❓ **Plot the training and validation losses through training**

In [ ]:
# Plot below your train/val loss history
# YOUR CODE HERE
# YOUR CODE HERE
# YOUR CODE HERE


# Run also this code to save figure as jpg in path below (it's your job to ensure it works)
fig = plt.gcf()
plt.savefig("tests/history.png")

❓ **Save your accuracy on test set as `score_test`**

In [ ]:
# YOUR CODE HERE

🧪 **Send your results below**

In [ ]:
from nbresult import ChallengeResult

result = ChallengeResult(
    "network",
    loss = model.loss,
    input_shape = list(model.input.shape),
    layer_names = [layer.name for layer in model.layers],
    final_activation = model.layers[-1].activation.__wrapped__._keras_api_names[0],
    score_baseline = score_baseline,
    score_test = score_test,
)
result.write()